In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

> **Data organizing and standardizing**

In [ ]:
countries_dict = {} #dict to contain all countries DataFrames
standard = [ #standard for the field names for all countries
    'total_tested', #Number of all tests performed in the counrtry - positive or negative
    'newly_tested', #Number of tests performed THIS DAY in the counrtry - positive or negative
    'total_swabed', #Number of all swabs taken
    'total_positive', #Number of all positive test outcomes in the counrtry
    'newly_positive', #Number of positive test outcomes in the counrtry FROM THIS DAY
    'total_negative', #Number of all negative test outcomes in the counrtry
    'newly_negative', #Number of negative test outcomes in the counrtry FROM THIS DAY
    'tests_in_progress', #Number of swabs taken and waiting to finish testing
    'total_active', #Number of patients currently infected
    'total_ICU', #Number of patients currently in ICU
    'total_dead', #Number of people who have died from COVID-19
    'newly_dead', #Number of people who have died from COVID-19 THIS DAY
    'total_recovered', #Number of people who have recovered from COVID-19
    'newly_recovered', #Number of people who have recovered from COVID-19 THIS DAY
    'total_home_confinement', #בידוד בית
    'total_hospitalized', # (לא ICU) כמות החולים באשפוז בבית חולים
    'newly_hospitalized',
    'totaL_vent' #מונשמים
]

standard_to_legend_dict = {
    'newly_positive':'No. of Positive Results',
    'newly_dead':'No. of new deaths'
}


In [ ]:
def drop_duplicate_columns(df):
#     print(df)

    return df.loc[:,~df.columns.duplicated()]

In [ ]:
for i in standard:
    print(i)

> > World wide

In [ ]:
def load_worldwide():
    world = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv').dropna(how='all', axis='columns')
    # print(world)
    grouped_world = world.groupby(by = 'Country/Region')
    
    # print(grouped_world.groups)
    for name, group in grouped_world:
#         world_countries[name] = group
        group = group.rename(columns = {'ObservationDate':'date', 'total':'total_swabed', 'Confirmed':'total_positive',
                            'Recovered':'total_recovered', 'Deaths':'total_dead'})
        group = group.drop(columns = ['SNo', 'Province/State', 'Country/Region', 'Last Update'])
        group = group.groupby(by = 'date').sum()

        group.index = pd.to_datetime(group.index).strftime('%Y-%m-%d')
        group['newly_dead'] = group['total_dead'].diff()
        group['newly_positive'] = group['total_positive'].diff()
        group['newly_recovered'] = group['total_recovered'].diff()
        countries_dict[name] = group
    print('world wide loaded')
load_worldwide()

> > Korea  

In [ ]:
def load_korea():
    korea = pd.read_csv('/kaggle/input/korea-covid19-tests/korea_covid19_tests - 1 (3).csv')
#     list(korea.columns)
    korea.index = korea['date']
    korea.index = pd.to_datetime(korea.index).strftime('%Y-%m-%d')
    del korea['date']
#     print(korea.index)
    # print(standard)
#     korea.info()

    
    
    korea = korea.rename(columns = {'total':'total_swabed', 'Confirmed_cases_total':'total_positive',
                            'discharged':'total_recovered', 'isolated':'total_active',
                            'Deceased':'total_dead', 'being_tested':'tests_in_progress', 'tested_negative':'total_negative'})

    korea['total_tested'] = korea['total_positive'] + korea['total_negative']
    korea['newly_tested'] = korea['total_tested'].diff()
    korea['newly_negative'] = korea['total_negative'].diff()    
    korea['newly_dead'] = korea['total_dead'].diff()
    korea['newly_recovered'] = korea['total_recovered'].diff()
    korea['newly_positive'] = korea['total_positive'].diff()
    countries_dict['South Korea'] = pd.concat([countries_dict['South Korea'], korea], axis=1, sort = True)
    countries_dict['South Korea'] = drop_duplicate_columns(countries_dict['South Korea'])
#     countries_dict['Korea'].info()
    print('korea loaded')
load_korea()

> > Italy

In [ ]:
def load_italy():
    italy = pd.read_csv("../input/covid19-in-italy/covid19_italy_region.csv")
    italy = italy.groupby(by = 'Date').sum()
#     print(standard)
#     italy.info()

    del italy['SNo'], italy['RegionCode'], italy['Latitude'], italy['Longitude']
    italy = italy.rename(columns = {'TotalHospitalizedPatients':'total_hospitalized', 'Deaths':'total_dead',
                                    'IntensiveCarePatients':'total_ICU', 'CurrentPositiveCases':'total_active',
                                    'Recovered':'total_recovered', 'TestsPerformed':'total_tested',
                                    'TotalPositiveCases':'total_positive','NewPositiveCases':'newly_positive',
                                    'HomeConfinement':'total_home_confinement'})

    italy['total_negative'] = italy['total_tested'] - italy['total_positive']
    italy['newly_tested'] = italy['total_tested'].diff()
    italy['newly_negative'] = italy['total_negative'].diff()
    italy['newly_dead'] = italy['total_dead'].diff()
    italy['newly_recovered'] = italy['total_recovered'].diff()
    italy.index = pd.to_datetime(italy.index).strftime('%Y-%m-%d')
    countries_dict['Italy'] = pd.concat([italy, countries_dict['Italy']], axis=1, sort=True)
    countries_dict['Italy'] = drop_duplicate_columns(countries_dict['Italy'])
    #c = c[~c.columns.duplicated(keep='first')]

#     print(italy.index)
#     countries_dict['Italy'].info()
    
    print('italy loaded')
load_italy()

> > USA

In [ ]:
def load_usa():
    usa = pd.read_csv("../input/covid19-in-usa/us_covid19_daily.csv")
    
    usa.index = [pd.to_datetime(date, format ='%Y%m%d').strftime('%Y-%m-%d') for date in usa['date']]
    
    del usa['date']
#     print(standard)
#     usa.info()


    usa = usa.rename(columns = {'positive':'total_positive', 'death':'total_dead', 'deathIncrease':'newly_dead', 'positiveIncrease':'newly_positive',
                                'total':'total_swabed', 'negative':'total_negative', 'totalTestResults':'total_tested', 'negativeIncrease':'newly_negative', 'totalTestResultsIncrease':'newly_tested',
                                'pending':'tests_in_progress', 'total':'total_swabed', 'hospitalized':'total_hospitalized', 'hospitalizedIncrease':'newly_hospitalized'})

    del usa['states'], usa['posNeg'], usa['dateChecked']
    
    countries_dict['US'] = pd.concat([countries_dict['US'], usa], axis=1, sort = True)
    countries_dict['US'] = drop_duplicate_columns(countries_dict['US'])

#     countries_dict['US'].info()
    print('usa loaded')
load_usa()

> > UK

In [ ]:
def load_uk():
    uk = pd.read_csv("/kaggle/input/covid19uk/data/covid-19-totals-uk.csv")
    uk.index = uk['Date']
#     print(uk.index)
    del uk['Date']
#     print(standard)
#     uk.info()


    uk = uk.rename(columns = {'ConfirmedCases':'total_positive', 'Deaths':'total_dead',
                                'Tests':'total_tested'})

    uk['total_negative'] = uk['total_tested'] - uk['total_positive']
    uk['newly_tested'] = uk['total_tested'].diff()
    uk['newly_negative'] = uk['total_negative'].diff()
    uk['newly_positive'] = uk['total_positive'].diff()    
    uk['newly_dead'] = uk['total_dead'].diff()

    countries_dict['UK'] = pd.concat([uk, countries_dict['UK']], axis=1, sort = True)
    countries_dict['UK'] = drop_duplicate_columns(countries_dict['UK'])
#     countries_dict['UK'].info()
    print('uk loaded')
load_uk()

> > pakistan

In [ ]:
def load_pakistan(): # not finished
    pak = pd.read_csv("/kaggle/input/corona-cases-in-pakistan/Corona Pakistan.csv")
    
#     print(standard)
    
    del pak['SNo'], pak['Global Cases'], pak['Global Deaths'], pak['Total Deaths in Italy'], pak['Total Cases in Italy'], pak['Returnees from China'],\
                pak['Returnees from Iran'], pak['Cumulative Travellers Screened'], pak['Cumulative Calls'], pak['New Calls'],\
                pak['Cumulative - Suspected Cases'], pak['New(Last 24 Hours) - Suspected Cases'], pak['Expired']
    pak = pak.rename(columns = {'Reporting Date':'date', 'Cumulative Tests Performed':'total_tested', 'Cumulative Test Positive Cases':'total_positive',
                                'Cumulative Tests Performed in Last 24 Hours':'newly_tested', 'Discharged':'total_recovered'})
    pak['newly_positive'] = pak['total_positive'].diff()
    pak['newly_recovered'] = pak['total_recovered'].diff()


    pak.index = pak['date']
    pak.index = pd.to_datetime(pak.index).strftime('%Y-%m-%d')
#     print(pak.index)
    del pak['date']
#     print(pak.info())
    print('loaded pakistan')
    countries_dict['Pakistan'] = pak
load_pakistan()  

>> spain

In [ ]:
def load_spain():
    spain = pd.read_csv("/kaggle/input/covid19spain/covid19_Spain.csv")
    spain = spain.groupby(by = 'Date').sum()
#     print(standard)

    spain = spain.rename(columns = {'Total Infected':'total_positive', 'Death':'total_dead',
                                'UCI':'total_ICU', 'Hospitalized':'total_hospitalized'})
    del spain['Report'], spain['IA']
    spain['newly_dead'] = spain['total_dead'].diff()
    spain['newly_positive'] = spain['total_positive'].diff()
    countries_dict['Spain'] = pd.concat([spain, countries_dict['Spain']], axis=1, sort = True)
#     print(spain.index)
    print('spain loaded')
#     print(spain.info())

# load_spain()

>>Brazil

In [ ]:
def load_brazil():
    brazil = pd.read_csv("/kaggle/input/corona-virus-brazil/brazil_covid19.csv")
#     print(brazil)
#     del brazil['hour']
    brazil = brazil.groupby(by = 'date').sum()
#     print(standard)
#     brazil.info()
    
    brazil = brazil.rename(columns = {'cases':'total_positive', 'deaths':'total_dead'})
#     print(brazil)
    
    countries_dict['Brazil'] = pd.concat([countries_dict['Brazil'], brazil], axis=1, sort = True)
    countries_dict['Brazil'] = drop_duplicate_columns(countries_dict['Brazil'])
#     brazil.info()
    
load_brazil()

>> swiss

In [ ]:
def load_swiss():
    swiss = pd.read_csv("/kaggle/input/openzh-covid-19/COVID19_Fallzahlen_CH_total.csv")
    #['total_tested', 'newly_tested', 'total_swabed', 'total_positive', 'newly_positive', 'total_negative', 'newly_negative', 'tests_in_progress',
    # 'total_active', 'total_ICU', 'total_dead', 'newly_dead', 'total_recovered', 'newly_recovered', 'total_home_confinement', 'total_hospitalized', 'newly_hospitalized']

    del swiss['time'], swiss['abbreviation_canton_and_fl']
    swiss = swiss.groupby(by = 'date').sum()
    
    swiss = swiss.rename(columns = {'ncumul_tested':'total_tested', 'ncumul_conf':'total_positive', 'ncumul_hosp':'total_hospitalized', 'ncumul_ICU':'total_ICU', 'ncumul_vent':'total_vent', 'ncumul_released':'total_recovered', 'ncumul_deceased':'total_dead'})
    
    swiss['total_negative'] = swiss['total_tested'] - swiss['total_positive']
    swiss['newly_tested'] = swiss['total_tested'].diff()
    swiss['newly_negative'] = swiss['total_negative'].diff()
    swiss['newly_positive'] = swiss['total_positive'].diff()    
    swiss['newly_dead'] = swiss['total_dead'].diff()
    swiss['newly_hospitalized'] = swiss['total_hospitalized'].diff()
    countries_dict['Switzerland'] = pd.concat([countries_dict['Switzerland'], swiss], axis=1, sort = True)
    countries_dict['Switzerland'] = drop_duplicate_columns(countries_dict['Switzerland'])
#     brazil.info()
    
load_swiss()

>>france

In [ ]:
def load_france():
    france = pd.read_csv("/kaggle/input/coronavirusdataset-france/chiffres-cles.csv")
    #['total_tested', 'newly_tested', 'total_swabed', 'total_positive', 'newly_positive', 'total_negative', 'newly_negative', 'tests_in_progress',
    # 'total_active', 'total_ICU', 'total_dead', 'newly_dead', 'total_recovered', 'newly_recovered', 'total_home_confinement', 'total_hospitalized', 'newly_hospitalized']

    del swiss['time'], swiss['abbreviation_canton_and_fl']
    swiss = swiss.groupby(by = 'date').sum()
    
    swiss = swiss.rename(columns = {'ncumul_tested':'total_tested', 'ncumul_conf':'total_positive', 'ncumul_hosp':'total_hospitalized', 'ncumul_ICU':'total_ICU', 'ncumul_vent':'total_vent', 'ncumul_released':'total_recovered', 'ncumul_deceased':'total_dead'})
    
    swiss['total_negative'] = swiss['total_tested'] - swiss['total_positive']
    swiss['newly_tested'] = swiss['total_tested'].diff()
    swiss['newly_negative'] = swiss['total_negative'].diff()
    swiss['newly_positive'] = swiss['total_positive'].diff()    
    swiss['newly_dead'] = swiss['total_dead'].diff()
    swiss['newly_hospitalized'] = swiss['total_hospitalized'].diff()
    countries_dict['Switzerland'] = pd.concat([countries_dict['Switzerland'], swiss], axis=1, sort = True)
    countries_dict['Switzerland'] = drop_duplicate_columns(countries_dict['Switzerland'])
    
load_france()

>>Load all

In [ ]:
def load_all():
    load_worldwide()
    load_korea()
    load_usa()
    load_italy()
    load_uk()
#     load_spain()
    load_pakistan()
    load_brazil()
    load_swiss()
    
load_all()

> **Vizualisations**

> > functios

In [ ]:
def check_update_date(countries):
    for country in countries:
        print(country, countries[country].index[-1])

In [ ]:
def check_avilable_data(countries):
    df = pd.DataFrame(index = standard)
    for country in countries:
        ones = {}
        for i in countries[country].columns:
            ones[i] = 1
        ser = pd.Series(ones)
        ser = ser.rename(country)
        df[country]=ser
    df.fillna(value=0, inplace=True)
    df = df.astype(int)
    df = df.style.background_gradient(cmap='Blues')
    return df

In [ ]:
check_avilable_data(countries_dict)

In [ ]:
pd.to_datetime(countries_dict['US'].index, )

In [ ]:
def masterViz(country_name=countries_dict, viz=1, country_df=None):
    if type(country_name) == list:
        for country in country_name:
            masterViz(country, viz)
    elif type(country_name) == dict:
        for country, di in country_name.items():
            masterViz(country, viz)
    else:
        functions = {
            1 : viz1, 'positives ratio' : viz1,
            2 : viz2, 'swabs' : viz2,
            3 : viz3, 'tests' : viz3
        }
        if country_df == None:
            country_df = countries_dict[country_name]
        try:
            functions[viz](country_name, country_df)
        except KeyError:
            print('no such function')

In [ ]:
def vizTest(country_name, fields, axes=plt.subplots(nrows=1, ncols=1), figsize=(20,10)):
    ofigsize = figsize
    figsize = (figsize[0]*axes.get_geometry()[0], figsize[1]*axes.get_geometry()[1])
    data = countries_dict[country_name][fields].sort_index()
    ax = data.plot.area(figsize=figsize, linewidth=3, title = country_name, fontsize = figsize[0]/axes.get_geometry()[0], stacked=False, ax=axes)
    ax.set_title(country_name, fontsize=15)
#     ax.set_yscale('log')
    ax.legend([standard_to_legend_dict[field] for field in fields], loc='upper left')
    ax.grid()
#     ax.set_xticks(fontsize=10)
    ax.set_xlabel('Date', fontsize = figsize[0]/(1.5*axes.get_geometry()[0]))
    ax.tick_params(axis='both', which='major', labelsize=figsize[0]/(2*axes.get_geometry()[0]))
#     print(data)
    return ax
        


In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2)
fig.tight_layout(pad=0.3)
vizTest('Mainland China', ['newly_positive', 'newly_dead'], axes[0,0])
vizTest('Singapore', ['newly_positive', 'newly_dead'], axes[0,1])
vizTest('Hong Kong', ['newly_positive', 'newly_dead'], axes[1,0])
vizTest('Taiwan', ['newly_positive', 'newly_dead'], axes[1,1])
vizTest('Japan', ['newly_positive', 'newly_dead'], axes[2,0])
vizTest('South Korea', ['newly_positive', 'newly_dead'], axes[2,1])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
vizTest('Mainland China', ['newly_positive', 'newly_dead'], axes, figsize=(20,10))
fig, axes = plt.subplots(nrows=1, ncols=1)
vizTest('Singapore', ['newly_positive', 'newly_dead'], axes, figsize=(20,10))
fig, axes = plt.subplots(nrows=1, ncols=1)
vizTest('Hong Kong', ['newly_positive', 'newly_dead'], axes, figsize=(20,10))
fig, axes = plt.subplots(nrows=1, ncols=1)
vizTest('Taiwan', ['newly_positive', 'newly_dead'], axes, figsize=(20,10))
fig, axes = plt.subplots(nrows=1, ncols=1)
vizTest('Japan', ['newly_positive', 'newly_dead'], axes, figsize=(20,10))
fig, axes = plt.subplots(nrows=1, ncols=1)
vizTest('South Korea', ['newly_positive', 'newly_dead'], axes, figsize=(20,10))